In [2]:
from pyathena import connect
from pyathena.pandas.util import as_pandas
from __future__ import print_function

# Import libraries
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import boto3
from botocore.client import ClientError
# below is used to print out pretty pandas dataframes
from IPython.display import display, HTML
%matplotlib inline

s3 = boto3.resource('s3')
client = boto3.client("sts")
account_id = client.get_caller_identity()["Account"]

my_session = boto3.session.Session()
region = my_session.region_name
athena_query_results_bucket = 'aws-athena-query-results-'+account_id+'-'+region


try:
    s3.meta.client.head_bucket(Bucket=athena_query_results_bucket)
except ClientError:
    bucket = s3.create_bucket(Bucket=athena_query_results_bucket)
    print('Creating bucket '+athena_query_results_bucket)
cursor = connect(s3_staging_dir='s3://'+athena_query_results_bucket+'/athena/temp').cursor()

In [ ]:
newd = {}
for i in range(len(microbio)):
    if microbio.org_name[i] in newd:
        if microbio.ab_name[i] in newd[microbio.org_name[i]]:
            newd[microbio.org_name[i]][microbio.ab_name[i]]+= microbio.interpretation[i]
        else:
            newd[microbio.org_name[i]][microbio.ab_name[i]] =  microbio.interpretation[i]
    #str(newtable.isolate_num[i]) in newd[newtable.org_name[i]]:
        #newd[newtable.org_name[i]][str(newtable.isolate_num[i])][newtable.ab_name[i]] = newtable.interpretation[i]
    #else:
        #newd[newtable.org_name[i]][str(newtable.isolate_num[i])] = {newtable.ab_name[i]: newtable.interpretation[i]}
    else:
        newd[microbio.org_name[i]] = {microbio.ab_name[i]: microbio.interpretation[i]}
all_abs = set()
for org in newd:
    for ab in newd[org]:
        all_abs.add(ab)
        val = newd[org][ab]
        resistivity = 0
        for i in val:
            if (i == 'R' or i == 'I'):
                resistivity += 1
        total = resistivity/len(val)
        newd[org][ab] = total
        #print(org, '->', ab, '->', newd[org][ab])

In [ ]:
# Getting the subject ids of all patients over 18 with an order placed within 24 hours of arrival 
#patients with susceptibility testing
query = """SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join
            mimiciii.microbiologyevents me
            on admit.subject_id = me.subject_id
            WHERE (me.org_name<> 'NA' AND me.ab_name<> 'NA'
            AND (Hour(me.charttime) - Hour(admit.admittime)) < 25)
            """
query2 = """SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join 
            mimiciii.patients p
            on admit.subject_id = p.subject_id
            WHERE ((Year(admit.admittime) - Year(p.dob)) > 17)
            """
fullquery = """
        WITH orderin24 AS(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join
            mimiciii.microbiologyevents me
            on admit.subject_id = me.subject_id
            WHERE (me.org_name<> 'NA' AND me.ab_name<> 'NA'
            AND (Hour(me.charttime) - Hour(admit.admittime)) < 25)
        ),
        over18 as(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join 
            mimiciii.patients p
            on admit.subject_id = p.subject_id
            WHERE ((Year(admit.admittime) - Year(p.dob)) > 17)
        ),
        cohort as( 
        SELECT DISTINCT o.subject_id
        FROM orderin24 o
        inner join
        over18 
        on o.subject_id = over18.subject_id
        )
        SELECT me.*
        FROM mimiciii.microbiologyevents me
        inner join
        cohort c
        on c.subject_id = me.subject_id
"""


cursor.execute(query)
orderin24 = as_pandas(cursor)
cursor.execute(query2)
over18 = as_pandas(cursor)
cohort = pd.merge(orderin24,over18, on = 'subject_id')
ast = pd.merge(microbio,cohort, on = 'subject_id')

In [ ]:
fullcohortquery = """
        WITH orderin24 AS(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join
            mimiciii.microbiologyevents me
            on admit.subject_id = me.subject_id
            WHERE ((Hour(me.charttime) - Hour(admit.admittime)) < 25)
        ),
        over18 as(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join 
            mimiciii.patients p
            on admit.subject_id = p.subject_id
            WHERE ((Year(admit.admittime) - Year(p.dob)) > 17)
        ),
        cohort as( 
        SELECT DISTINCT o.subject_id
        FROM orderin24 o
        inner join
        over18 
        on o.subject_id = over18.subject_id
        )
        SELECT me.*
        FROM mimiciii.microbiologyevents me
        inner join
        cohort c
        on me.subject_id = c.subject_id
        WHERE (me.org_name<> 'NA' AND me.ab_name<> 'NA')
"""
cursor.execute(fullcohortquery)
full = as_pandas(cursor)

In [ ]:
print("Number of subjects in cohort with ast:", len(full['subject_id'].unique()))
print("Number of hospital admissions in cohort with ast:", len(full.hadm_id.unique()))

Using Stanford data we trained 12 binary machine learning models to estimate the probability that common antibiotic selections would provide activity against infections at the point in time empiric antibiotics were chosen. An antibiotic selection was said to provide activity against a patient’s infection if all microbial organisms that grew in the patient’s microbial cultures were listed as susceptible to at least one of the antibiotics in the selection.

Feature Matrix:
- construct with data up until the prediction time. (time empiric antibiotic were ordered) 
- categorical features
    - diagnosis codes
    - procedure orders
    - lab orders
    - medication orders
    - imaging orders
    - orders for respiratory care
    - for prior microbial culture resutls, categorical features were constructed based on the antibiotic susceptibility patterns of extracteed isolates 
    - 

In [ ]:
statictable = """
        SELECT DISTINCT mimiciii.admissions.subject_id, mimiciii.admissions.hadm_id,
        mimiciii.patients.gender, mimiciii.patients.dob, 
        mimiciii.admissions.insurance,
        mimiciii.admissions.language, mimiciii.admissions.religion,
        mimiciii.admissions.marital_status, mimiciii.admissions.ethnicity
        FROM 
        mimiciii.admissions, mimiciii.patients
        limit 50
"""
cursor.execute(statictable)
demographic = as_pandas(cursor)

In [ ]:
demographic.head(3)

In [ ]:
demographic.to_csv("C:\\Users\\brokowst\\Desktop\\demographic.csv", index=False)

In [ ]:
os.getcwd()

In [ ]:
ids = full.subject_id.unique().tolist()

In [ ]:
print(admissions['subject_id'].unique().tolist()[:25])

In [ ]:
merged = pd.merge(admissions, full)

In [ ]:
charteventsquery = """
        WITH orderin24 AS(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join
            mimiciii.microbiologyevents me
            on admit.subject_id = me.subject_id
            WHERE ((Hour(me.charttime) - Hour(admit.admittime)) < 25)
        ),
        over18 as(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join 
            mimiciii.patients p
            on admit.subject_id = p.subject_id
            WHERE ((Year(admit.admittime) - Year(p.dob)) > 17)
        ),
        cohort as( 
        SELECT DISTINCT o.subject_id
        FROM orderin24 o
        inner join
        over18 
        on o.subject_id = over18.subject_id
        ),
        ast as(SELECT me.*
        FROM mimiciii.microbiologyevents me
        inner join
        cohort c
        on me.subject_id = c.subject_id
        WHERE (me.org_name<> 'NA' AND me.ab_name<> 'NA'))
        SELECT distinct
        ce.subject_id, ce.hadm_id, ce.icustay_id, ce.charttime, ce.itemid
        adm.admittime, adm.ethnicity, adm.insurance
        FROM mimiciii.chartevents ce
        inner join ast
        ON 
        ast.subject_id = ce.subject_id
"""
cursor.execute(charteventsquery)
ce = as_pandas(cursor)

In [ ]:
ce

In [ ]:
merged = pd.merge(admissions, ce, on = ["subject_id, hadm_id"]
                  

In [ ]:
#To start and build an example table build one where subject id = 36 
examplepersonalmatrix = """

SELECT adm.row_id, adm.subject_id, adm.hadm_id,
adm.admittime, adm.dischtime, adm.deathtime, 
adm.admission_type, adm.admission_location, adm.discharge_location,
adm.edregtime, adm.edouttime, adm.diagnosis,
ce.charttime, ce.storetime, 
ce.cgid, mimiciii.d_items.label, ce.value, ce.valuenum
FROM mimiciii.admissions adm,
mimiciii.chartevents ce, mimiciii.d_items
WHERE adm.subject_id = 36 AND ce.subject_id = 36 
AND mimiciii.d_items.itemid = ce.itemid
limit 50
"""
cursor.execute(examplepersonalmatrix)
s = as_pandas(cursor)
s.head(10)

In [ ]:
s.to_csv("patient36.csv")

In [ ]:
len(s)

In [ ]:
patient = {}

#subjectid #hadm_id #timeofinput #typeofinput #detailsofinput (input id) 

#ex 

#main table
#subjectid #hadm_id #timeofinput #typeofinput #detailsofinput (input id)
# 36       |10256  |  time      | chart_events| ce1023 
# 36       |10256  | time   |   microbiology_events | me1034 

#ce table
#id #label #value #valuenum #etc 
#ce1023 | o2flow | 15 | 15.00

#


#use the inputid to link to the table with the details of that input

#details of input change based on type of input 
#different types of input 
    #details of input
    
#demographic --> nested stamped with time of admission
    #age
    #language
    #religion
    #marital status
    #ethnicity
#admit --> nested stamped with time of admission
    #admission type
    #admission location
#discharge --> nested stamped with time of discharge
    #dischargetime
#chartevents --> nested stamped with charttime
    # itemid --> (vital readings) code defined in d_items.label
    #value for that item
    #valuenum
    #valueom
    #warning
    #error
    #resultsstatus
#death --> stamped with time of death or n/a
    

            #itemid --> (readings) code defined in d_items.label
                #value for that itemid
                #valueom 
                #warning
                #error
                #cgid
        #storetime
        #edouttime
        #dischargetime
            #dischargelocation
        

for i in range(len(s)):
    

In [ ]:
#Want to create a dictionary that has temporal based information
for i in range(len(microbio)):
    if microbio.org_name[i] in newd:
        if microbio.ab_name[i] in newd[microbio.org_name[i]]:
            newd[microbio.org_name[i]][microbio.ab_name[i]]+= microbio.interpretation[i]
        else:
            newd[microbio.org_name[i]][microbio.ab_name[i]] =  microbio.interpretation[i]
    #str(newtable.isolate_num[i]) in newd[newtable.org_name[i]]:
        #newd[newtable.org_name[i]][str(newtable.isolate_num[i])][newtable.ab_name[i]] = newtable.interpretation[i]
    #else:
        #newd[newtable.org_name[i]][str(newtable.isolate_num[i])] = {newtable.ab_name[i]: newtable.interpretation[i]}
    else:
        newd[microbio.org_name[i]] = {microbio.ab_name[i]: microbio.interpretation[i]}
all_abs = set()
for org in newd:
    for ab in newd[org]:
        all_abs.add(ab)
        val = newd[org][ab]
        resistivity = 0
        for i in val:
            if (i == 'R' or i == 'I'):
                resistivity += 1
        total = resistivity/len(val)
        newd[org][ab] = total
        print(org, '->', ab, '->', newd[org][ab])


In [ ]:
#Get Info from admissions per subject_id

#initial patient data query 

admissionsquery = """
        WITH orderin24 AS(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join
            mimiciii.microbiologyevents me
            on admit.subject_id = me.subject_id
            WHERE ((Hour(me.charttime) - Hour(admit.admittime)) < 25)
        ),
        over18 as(
        SELECT distinct admit.subject_id
            FROM mimiciii.admissions admit
            inner join 
            mimiciii.patients p
            on admit.subject_id = p.subject_id
            WHERE ((Year(admit.admittime) - Year(p.dob)) > 17)
        ),
        cohort as( 
        SELECT DISTINCT o.subject_id
        FROM orderin24 o
        inner join
        over18 
        on o.subject_id = over18.subject_id
        ),
        ast as(SELECT me.*
        FROM mimiciii.microbiologyevents me
        inner join
        cohort c
        on me.subject_id = c.subject_id
        WHERE (me.org_name<> 'NA' AND me.ab_name<> 'NA'))
        SELECT DISTINCT adm.subject_id, adm.hadm_id, adm.admittime, adm.ethnicity, adm.insurance
        FROM mimiciii.admissions adm
        inner join ast
        ON ast.subject_id = adm.subject_id
"""
cursor.execute(admissionsquery)
admissions = as_pandas(cursor)

print("Number of subjects in cohort with ast:", len(admissions['subject_id'].unique()))
print(admissions['subject_id'].unique().tolist()[:25])

In [3]:
#main table
#subjectid #hadm_id #timeofinput #typeofinput #detailsofinput (input id)
# 36       |10256  |  time      | chart_events| ce1023 
# 36       |10256  | time   |   microbiology_events | me1034 
#in here
    #staticpatientinfo
        # gender, dob, dod, insurance, language, religion,
        # marital_status, ethnicity
    #admissioninfo
        #
    #labtests
        #chartevents
        #labevents 
        #microbiologyevents
    #medication 
        #perscriptions -> done 
        #input_events_cv -> need to do 
        # input_events_mv -> need to do
    #outputs 
    #clinicalnotes
    #diagnosis and procedures 
#feature matrix query
query = """
with combined as 
(SELECT adm.subject_id, adm.hadm_id, 
adm.admittime as time,
'admit' as infotype, 'admission_type' as label, adm.admission_type as value,
adm.row_id as id
FROM mimiciii.admissions adm

UNION 
SELECT adm.subject_id, adm.hadm_id, 
adm.admittime as time, 
'admit' as infotype, 'admission_location' as label, adm.admission_location as value,
adm.row_id as id
FROM mimiciii.admissions adm

UNION 
SELECT adm.subject_id, adm.hadm_id, 
adm.dischtime as time, 
'disch' as infotype, 'discharge_location' as label, adm.discharge_location as value,
adm.row_id as id
FROM mimiciii.admissions adm

UNION 
SELECT ce.subject_id, ce.hadm_id,
ce.charttime as time,
'chart events' as infotype, items.label as label, ce.value as value,
ce.row_id as id
FROM mimiciii.chartevents ce, mimiciii.d_items items
WHERE ce.itemid = items.itemid

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'drug_type' as infotype,  pers.drug_type as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'drug' as infotype,  pers.drug as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.enddate as time,
'perscriptions' as infotype, 'enddate' as infotype,  'na' as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'formulary_drug_cd' as infotype,  pers.formulary_drug_cd as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'gsm' as infotype,  pers.gsn as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'ndc' as infotype,  pers.ndc as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'prod_strength' as infotype,  pers.prod_strength as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'dose_val_rx' as infotype,  pers.dose_val_rx as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'dose_unit_rx' as infotype,  pers.dose_unit_rx as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'form_val_disp' as infotype,  pers.form_val_disp as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'form_unit_disp' as infotype,  pers.form_unit_disp as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, 'route' as infotype,  pers.route as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT me.subject_id, me.hadm_id,
me.charttime as time,
'microbiologyevents' as infotype, 'spec_type_desc' as infotype,  me.spec_type_desc as value,
me.row_id as id
FROM mimiciii.microbiologyevents me

UNION 
SELECT me.subject_id, me.hadm_id,
me.charttime as time,
'microbiologyevents' as infotype, 'org_name' as infotype,  me.org_name as value,
me.row_id as id
FROM mimiciii.microbiologyevents me

UNION 
SELECT me.subject_id, me.hadm_id,
me.charttime as time,
'microbiologyevents' as infotype, 'isolate_num' as infotype,  CAST(me.isolate_num as varchar) as value,
me.row_id as id
FROM mimiciii.microbiologyevents me

UNION 
SELECT me.subject_id, me.hadm_id,
me.charttime as time,
'microbiologyevents' as infotype, 'ab_name' as infotype,  me.ab_name as value,
me.row_id as id
FROM mimiciii.microbiologyevents me

UNION 
SELECT me.subject_id, me.hadm_id,
me.charttime as time,
'microbiologyevents' as infotype, 'interpretation' as infotype,  me.interpretation as value,
me.row_id as id
FROM mimiciii.microbiologyevents me

UNION 
SELECT ne.subject_id, ne.hadm_id,
ne.chartdate as time,
'noteevents' as infotype, ne.category as infotype,  ne.text as value,
ne.row_id as id
FROM mimiciii.noteevents ne

UNION 
SELECT le.subject_id, le.hadm_id,
le.charttime as time,
'labevents' as infotype, 'fluid' as label, d_labitems.fluid as value,
le.row_id as id
FROM mimiciii.labevents le, mimiciii.d_labitems d_labitems
WHERE le.itemid = d_labitems.itemid

UNION 
SELECT le.subject_id, le.hadm_id,
le.charttime as time,
'labevents' as infotype, 'category' as label, d_labitems.category as value,
le.row_id as id
FROM mimiciii.labevents le, mimiciii.d_labitems d_labitems
WHERE le.itemid = d_labitems.itemid

UNION 
SELECT le.subject_id, le.hadm_id,
le.charttime as time,
'labevents' as infotype, 'loinc_code' as label, d_labitems.loinc_code as value,
le.row_id as id
FROM mimiciii.labevents le, mimiciii.d_labitems d_labitems
WHERE le.itemid = d_labitems.itemid

UNION 
SELECT le.subject_id, le.hadm_id,
le.charttime as time,
'labevents' as infotype, 'label' as label, d_labitems.label as value,
le.row_id as id
FROM mimiciii.labevents le, mimiciii.d_labitems d_labitems
WHERE le.itemid = d_labitems.itemid

UNION 
SELECT le.subject_id, le.hadm_id,
le.charttime as time,
'labevents' as infotype, 'value' as label, le.value as value,
le.row_id as id
FROM mimiciii.labevents le, mimiciii.d_labitems d_labitems
WHERE le.itemid = d_labitems.itemid

UNION 
SELECT le.subject_id, le.hadm_id,
le.charttime as time,
'labevents' as infotype, 'valueuom' as label, le.valueuom as value,
le.row_id as id
FROM mimiciii.labevents le, mimiciii.d_labitems d_labitems
WHERE le.itemid = d_labitems.itemid

UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'diagnoses' as infotype, 'short_title' as label, dicd.short_title as value,
di.row_id as id
FROM mimiciii.diagnoses_icd di, mimiciii.d_icd_diagnoses dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code

UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'diagnoses' as infotype, 'long_title' as label, dicd.long_title as value,
di.row_id as id
FROM mimiciii.diagnoses_icd di, mimiciii.d_icd_diagnoses dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code

UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'diagnoses' as infotype, 'seq_num' as label, CAST(di.seq_num as varchar) as value,
di.row_id as id
FROM mimiciii.diagnoses_icd di, mimiciii.d_icd_diagnoses dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code


UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'procedures' as infotype, 'short_title' as label, dicd.short_title as value,
di.row_id as id
FROM mimiciii.procedures_icd di, mimiciii.d_icd_procedures dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code

UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'procedures' as infotype, 'long_title' as label, dicd.long_title as value,
di.row_id as id
FROM mimiciii.procedures_icd di, mimiciii.d_icd_procedures dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code

UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'procedures' as infotype, 'seq_num' as label, CAST(di.seq_num as varchar) as value,
di.row_id as id
FROM mimiciii.procedures_icd di, mimiciii.d_icd_procedures dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code

UNION 
SELECT oe.subject_id, oe.hadm_id,
oe.charttime as time,
'outputevents' as infotype,  items.label as label, CONCAT(CAST(oe.value as varchar),' ', oe.valueuom) as value,
oe.row_id as id
FROM mimiciii.outputevents oe, mimiciii.d_items items
WHERE oe.itemid = items.itemid 

)
SELECT * FROM combined where subject_id IN (31321)
order by time, infotype

"""
cursor.execute(query)
s = as_pandas(cursor)
s

drop note events 
static
microbiology -> separate schema

perscription -> name and 1 
labevents -> name and value 
chartevents -> name and value 
diagnoses -> name and 1 
procedures -> name and 1 

if s 1 if i or r 0 
rows -> all patient info before first drug perscribed
columsn -> wide df from above 
output -> antibiotic wide vector with 1 row of the 0 if i or r 1 if s 

pivot -> aggregate and go by most recent

SyntaxError: invalid syntax (<ipython-input-3-82bb075cb30d>, line 293)

In [ ]:
staticquery = """

with static as(
SELECT DISTINCT p.subject_id, p.gender, p.dob, p.dod, 
       adm.insurance, adm.language, adm.religion, 
       adm.marital_status, adm.ethnicity
from mimiciii.patients p, mimiciii.admissions adm
where p.subject_id = adm.subject_id)
SELECT * FROM static where where subject_id IN (31321)


"""
cursor.execute(staticquery)
static = as_pandas(cursor)

In [ ]:
intv = """
with intervention as(
select pers.hadm_id, pers.startdate, pers.enddate, pers.drug
from mimiciii.prescriptions pers
where (lower(pers.drug) like '%adoxa%' or 
       lower(drug) like '%ala-tet%' or
       lower(drug) like '%alodox%' or
       lower(drug) like '%amikacin%' or
       lower(drug) like '%amikin%' or
       lower(drug) like '%amoxicillin%' or
       lower(drug) like '%amoxicillin%clavulanate%'or 
       lower(drug) like '%clavulanate%'or
       lower(drug) like '%ampicillin%' or
       lower(drug) like '%augmentin%' or
       lower(drug) like '%avelox%'or
       lower(drug) like '%avidoxy%' or
       lower(drug) like '%azactam%'or
       lower(drug) like '%azithromycin%'or
       lower(drug) like '%aztreonam%' or
       lower(drug) like '%axetil%' or
       lower(drug) like '%bactocill%' or
       lower(drug) like '%bactrim%' or
       lower(drug) like '%bethkis%' or
       lower(drug) like '%biaxin%' or
       lower(drug) like '%bicillin l-a%'or
       lower(drug) like '%cayston%' or
       lower(drug) like '%cefazolin%'or
       lower(drug) like '%cedax%' or
       lower(drug) like '%cefoxitin%' or
       lower(drug) like '%ceftazidime%' or
       lower(drug) like '%cefaclor%' or
       lower(drug) like '%cefadroxil%' or
       lower(drug) like '%cefdinir%' or
       lower(drug) like '%cefditoren%'or
       lower(drug) like '%cefepime%'or
       lower(drug) like '%cefotetan%' or
       lower(drug) like '%cefotaxime%' or
       lower(drug) like '%cefpodoxime%' or
       lower(drug) like '%cefprozil%' or
       lower(drug) like '%ceftibuten%' or
       lower(drug) like '%ceftin%' or
       lower(drug) like '%cefuroxime %' or
       lower(drug) like '%cefuroxime%' or
       lower(drug) like '%cephalexin%' or
       lower(drug) like '%chloramphenicol%' or
       lower(drug) like '%cipro%' or
       lower(drug) like '%ciprofloxacin%' or
       lower(drug) like '%claforan%' or
       lower(drug) like '%clarithromycin%' or
       lower(drug) like '%cleocin%' or
       lower(drug) like '%clindamycin%' or
       lower(drug) like '%cubicin%'or
       lower(drug) like '%dicloxacillin%' or
       lower(drug) like '%doryx%' or
       lower(drug) like '%doxycycline%' or
       lower(drug) like '%duricef%' or
       lower(drug) like '%dynacin%'or
       lower(drug) like '%ery-tab%' or
       lower(drug) like '%eryped%' or
       lower(drug) like '%eryc%' or
       lower(drug) like '%erythrocin%' or
       lower(drug) like '%erythromycin%' or
       lower(drug) like '%factive%' or
       lower(drug) like '%flagyl%' or
       lower(drug) like '%fortaz%'or
       lower(drug) like '%furadantin%' or
       lower(drug) like '%garamycin%' or
       lower(drug) like '%gentamicin%' or
       lower(drug) like '%kanamycin%' or
       lower(drug) like '%keflex%' or
       lower(drug) like '%ketek%'or
       lower(drug) like '%levaquin%' or
       lower(drug) like '%levofloxacin%' or
       lower(drug) like '%lincocin%' or
       lower(drug) like '%macrobid%' or
       lower(drug) like '%macrodantin%' or
       lower(drug) like '%maxipime%' or
       lower(drug) like '%mefoxin%'or
       lower(drug) like '%metronidazole%' or
       lower(drug) like '%minocin%' or
       lower(drug) like '%minocycline%' or
       lower(drug) like '%monodox%' or
       lower(drug) like '%monurol%' or
       lower(drug) like '%morgidox%' or
       lower(drug) like '%moxatag%' or
       lower(drug) like '%moxifloxacin%'or
       lower(drug) like '%myrac%' or
       lower(drug) like '%nafcillin sodium%'or
       lower(drug) like '%nicazel doxy 30%' or
       lower(drug) like '%nitrofurantoin%' or
       lower(drug) like '%noroxin%' or
       lower(drug) like '%ocudox%' or
       lower(drug) like '%ofloxacin%' or
       lower(drug) like '%omnicef%' or
       lower(drug) like '%oracea%' or
       lower(drug) like '%oraxyl%'or
       lower(drug) like '%oxacillin%' or
       lower(drug) like '%pc pen vk%'or
      lower(drug) like '%pce dispertab%' or
       lower(drug) like '%panixine%' or
       lower(drug) like '%pediazole%' or
      lower(drug) like '%penicillin%' or
     lower(drug) like '%periostat%' or
     lower(drug) like '%pfizerpen%' or
       lower(drug) like '%piperacillin%' or
      lower(drug) like '%tazobactam%' or
       lower(drug) like '%primsol%' or
       lower(drug) like '%proquin%' or
      lower(drug) like '%raniclor%' or
      lower(drug) like '%rifadin%'or
       lower(drug) like '%rifampin%' or
       lower(drug) like '%rocephin%' or
       lower(drug) like '%smz-tmp%' or
       lower(drug) like '%septra%' or
      lower(drug) like '%septra ds%' or
       lower(drug) like '%septra%' or
      lower(drug) like '%solodyn%' or
    lower(drug) like '%spectracef%' or
      lower(drug) like '%streptomycin sulfate%' or
       lower(drug) like '%sulfadiazine%' or
       lower(drug) like '%sulfamethoxazole%' or
      lower(drug) like '%trimethoprim%' or
       lower(drug) like '%sulfatrim%' or
       lower(drug) like '%sulfisoxazole%' or
      lower(drug) like '%suprax%' or
       lower(drug) like '%synercid%' or
       lower(drug) like '%tazicef%' or
       lower(drug) like '%tetracycline%' or
       lower(drug) like '%timentin%' or
       lower(drug) like '%tobi%' or
     lower(drug) like '%tobramycin%' or
      lower(drug) like '%trimethoprim%' or
       lower(drug) like '%unasyn%' or
     lower(drug) like '%vancocin%' or
       lower(drug) like '%vancomycin%' or
       lower(drug) like '%vantin%' or
       lower(drug) like '%vibativ%' or
      lower(drug) like '%vibra-tabs%' or
       lower(drug) like '%vibramycin%' or
      lower(drug) like '%zinacef%' or
      lower(drug) like '%zithromax%' or
       lower(drug) like '%zmax%' or
       lower(drug) like '%zosyn%'or
       lower(drug) like '%zyvox%')
       and 
       pers.subject_id = 31321
       )
  Select hadm_id, drug, startdate as startdate, enddate as enddate from intervention 
  order by startdate asc
"""
cursor.execute(intv)
intv = as_pandas(cursor)
intv

#each perscription is a label --> query all info for each perscription to train

In [ ]:
#query to get all the information from the table above before the first antibiotic was ordered 
#Essentially, select * from s where s.date is before the perscription of the administered antibiotic
#first find out when the antibiotic was administered

s.to_csv("patient31321.csv")
intv.to_csv("intc31321.csv")

In [ ]:
#I have s and intv --> I want to take all values from s where the hadm_ids are equal
#and the dates are prior to the admission of the antibiotic 

#only problem is we dont have dates for diagnoses so the diagnoses are not incorporated

from pandasql import sqldf 
mysql = lambda q: sqldf(q, globals()) 
query = '''
with intv1 as(
select * from intv limit 3)
select distinct s.* from s 
left join intv1 on 
s.hadm_id = intv1.hadm_id
where s.time < intv1.startdate
'''
newtable = mysql(query)
newtable


In [ ]:
newtable.label.unique()

In [ ]:
microevents = '''
select * from newtable where infotype = 'microbiologyevents'
'''
micro = mysql(microevents)
micro

In [ ]:
pivot = micro.pivot(index = 'id', columns = 'label', values = 'value')

In [ ]:
pivot

In [ ]:
pivot.dropna()

If all organisms in a patients microbial order were S to at least one of the perscribed antibiotics in the selection (the antibiotic we are training the model for) 
prediction time is when the first antibiotic was ordered. 

perscription -> name and 1 
labevents -> name and value 
chartevents -> name and value 
diagnoses -> name and 1 
procedures -> name and 1 

if s 1 if i or r 0 
rows -> all patient info before first drug perscribed
columsn -> wide df from above 
output -> antibiotic wide vector with 1 row of the 0 if i or r 1 if s 




pivot -> aggregate and go by most recent



each input to the model is all the info in the ehr prior to the microbiology event



In [4]:
query = """

with combined as 
(
SELECT ce.subject_id, ce.hadm_id,
ce.charttime as time,
'chart events' as infotype, items.label as label, ce.value as value,
ce.row_id as id
FROM mimiciii.chartevents ce, mimiciii.d_items items
WHERE ce.itemid = items.itemid

UNION 
SELECT pers.subject_id, pers.hadm_id,
pers.startdate as time,
'perscriptions' as infotype, pers.drug as label, Cast(Day(pers.enddate -  pers.startdate) as varchar)as value,
pers.row_id as id
FROM mimiciii.prescriptions pers

UNION 
SELECT le.subject_id, le.hadm_id,
le.charttime as time,
'labevents' as infotype, d_labitems.label as label, le.value as value,
le.row_id as id
FROM mimiciii.labevents le, mimiciii.d_labitems d_labitems
WHERE le.itemid = d_labitems.itemid

UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'diagnoses' as infotype, dicd.short_title as label, cast(1 as varchar) as value,
di.row_id as id
FROM mimiciii.diagnoses_icd di, mimiciii.d_icd_diagnoses dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code

UNION 
SELECT di.subject_id, di.hadm_id,
adm.dischtime as time,
'procedures' as infotype, dicd.short_title as label, cast(1 as varchar) as value,
di.row_id as id
FROM mimiciii.procedures_icd di, mimiciii.d_icd_procedures dicd, mimiciii.admissions adm
WHERE di.subject_id = adm.subject_id AND di.icd9_code = dicd.icd9_code
)
SELECT * FROM combined where subject_id IN (31321, 45, 31320, 2100)
order by time, infotype

"""

microbioquery = """

SELECT me.subject_id, me.hadm_id,
me.charttime as time, me.org_name as organism, me.ab_name as antibiotic, CASE when
   me.interpretation = 'S' then 1 else 0 as interpretation
FROM mimiciii.microbiologyevents me
where subject_id IN (31321, 45, 31320, 2100) and me.interpretation is not null
"""



cursor.execute(query)
s = as_pandas(cursor)
s

,subject_id,hadm_id,time,infotype,label,value,id
0,31320,NaN,2117-08-10 14:00:00,labevents,Urea Nitrogen,37,19165191
1,31320,NaN,2117-08-10 14:00:00,labevents,"Cholesterol, LDL, Calculated",54,19165180
2,31320,NaN,2117-08-10 14:00:00,labevents,Thyroid Stimulating Hormone,3.2,19165189
3,31320,NaN,2117-08-10 14:00:00,labevents,RDW,14.8,19165203
4,31320,NaN,2117-08-10 14:00:00,labevents,Potassium,6.3,19165186
...,...,...,...,...,...,...,...
31573,2100,140146.0,2142-05-03 10:15:00,procedures,Portal contr phlebogram,1,174325
31574,2100,131956.0,2142-05-03 10:15:00,procedures,Oth endo proc oth vessel,1,59776
31575,2100,140146.0,2142-05-03 10:15:00,procedures,Sm bowel endoscopy NEC,1,174327
31576,2100,140146.0,2142-05-03 10:15:00,procedures,Angio oth non-coronary,1,174323


In [5]:
import warnings
warnings.filterwarnings('ignore')
k = s.drop(columns = ['id','infotype'])
k = k.pivot_table(index = ['subject_id', 'hadm_id', 'time'], 
        columns = 'label', values = 'value',
        aggfunc = 'first', fill_value = 0)
cols = k.columns
mod = True
todrop = set(())
for col in cols:
    #for each column get a list of unique values
    numb = True
    unique = k[col].unique()
    uniset = set(())
    for v in range(len(unique)):
        try:
            uniset.add(str(unique[v]).to_lower())
        except AttributeError:
            uniset.add(unique[v])
    for value in uniset:
        #for each of the unique values try to convert them to float, if we cant
        try:
            float(value)
        except ValueError:
            numb = False
            k[col+'('+value+')'] = np.where(k[col]==value,1,0)
            todrop.add(col)
            #if we cant conver to float then convert to category and one hot encode
    if(numb == True):
        k[col] = k[col].astype('float')
k = k.drop(columns = todrop)
k = k.reset_index()

In [6]:
len(k.columns)

1296

In [ ]:
k.columns

In [11]:
k.head()

label,subject_id,hadm_id,time,0.9% Sodium Chloride,1/2 NS,18 Gauge Dressing Occlusive,18 Gauge placed in outside facility,18 Gauge placed in the field,5% Dextrose,ABP Alarm [High],...,WBC(3-5),WBC(21-50),WBC(0-2),Warming [Device](Blanket),Warming [Status](On),Waveform-Vent(RAMP),Yeast(NONE),bath(done),bath(Done),bath(Refused)
0,45,157907.0,2129-06-09 18:19:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,45,157907.0,2129-06-09 18:20:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,45,157907.0,2129-06-09 18:24:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,45,157907.0,2129-06-09 19:34:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,45,157907.0,2129-06-09 20:48:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#for col in k.columns:
 #   print(col, k[col].unique())

In [ ]:
k.head()

In [9]:
microbioquery = """

SELECT me.subject_id, me.hadm_id,
me.charttime as time, me.org_name as organism, me.ab_name as antibiotic, 
CASE when
   me.interpretation = 'S' then 1 
else 0 end as interpretation
FROM mimiciii.microbiologyevents me
where subject_id IN (31321, 45, 31320, 2100) and me.interpretation is not null
"""
cursor.execute(microbioquery)
m = as_pandas(cursor)
m

,subject_id,hadm_id,time,organism,antibiotic,interpretation
0,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,CEFUROXIME,1
1,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,AMPICILLIN/SULBACTAM,1
2,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,CEFTAZIDIME,1
3,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,CEFEPIME,1
4,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,LEVOFLOXACIN,0
5,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,GENTAMICIN,1
6,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,NITROFURANTOIN,1
7,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,TRIMETHOPRIM/SULFA,1
8,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,CEFAZOLIN,1
9,2100,131956,2136-12-16 01:31:00,ESCHERICHIA COLI,AMPICILLIN,1


We will frame the supervised learning problem as predicting the microbiology susceptibility test results at the current time (t) given all prior ehr data.


microbioevents df = index = subject_id, hadm_id, chart_time, organism, columns = antibiotics




In [10]:
#I need to structure the target and training data
m = m.pivot_table(index = ['subject_id', 'hadm_id', 'time', 'organism'], 
        columns = 'antibiotic', values = 'interpretation',
        aggfunc = 'first', fill_value = 'na')
m

antibiotic                                                  AMPICILLIN  \
subject_id hadm_id time                organism                          
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI            1.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM        0.0   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +          na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +          na   

antibiotic                                                  AMPICILLIN/SULBACTAM  \
subject_id hadm_id time                organism                                    
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI                      1.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM                   na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +                    na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +                    na   

antibiotic                                                  CEFAZOLIN  \
subject_id hadm_id time                organism                         
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI           1.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM        na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +         na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +         na   

antibiotic                                                  CEFEPIME  \
subject_id hadm_id time                organism                        
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI          1.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM       na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +        na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +        na   

antibiotic                                                  CEFTAZIDIME  \
subject_id hadm_id time                organism                           
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI             1.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM          na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +           na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +           na   

antibiotic                                                  CEFTRIAXONE  \
subject_id hadm_id time                organism                           
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI             1.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM          na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +           na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +           na   

antibiotic                                                  CEFUROXIME  \
subject_id hadm_id time                organism                          
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI            1.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM         na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +          na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +          na   

antibiotic                                                  CIPROFLOXACIN  \
subject_id hadm_id time                organism                             
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI               0.0   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM            na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +             na   
                   2132-05-29 14:08:00 STAPH AUREUS COAG +             na   

antibiotic                                                  CLINDAMYCIN  \
subject_id hadm_id time                organism                           
2100       131956  2136-12-16 01:31:00 ESCHERICHIA COLI              na   
31320      154445  2117-09-12 14:20:00 ENTEROCOCCUS FAECIUM          na   
31321      136755  2132-05-28 08:38:00 STAPH AUREUS COAG +          

In [ ]:
from datetime import datetime
f=m.index[1][2].to_pydatetime()
print(f)
g = m.index[2][2].to_pydatetime()
print(f > g)



#for the input data if the subjectid = subjectid and date < preddate then use all of those rows as features

In [10]:
#format data -> for LTSM its a 3d input 
#(number of samples, number of timesteps, number of features)


#for each prediction
# the number of samples = length of m, 
#timesteps = rows with the same subject id and time before m
#features = the number of columns in k


#drop the time, subject id, and hadm id after structuring the problem


dfarr = []
for i in range(len(m)):
    df = pd.DataFrame(columns = k.columns)
    for j in range(len(k)):
        if k['subject_id'][j] == m.index[i][0] and k['time'][j].to_pydatetime() < m.index[i][2].to_pydatetime():
            df = df.append(k.iloc[j])
    dfarr.append(df.to_numpy())

In [11]:
dfarrc = np.asarray(dfarr)
for i in range(len(dfarrc)):
    print(dfarrc[i].shape)
max = 0
for i in range(len(dfarrc)):
    if(dfarrc[i].shape[0] > max):
        max = dfarrc[i].shape[0]
print(max)
#pad empty rows with 0 

paddedrows = []
for i in range(len(dfarrc)):
    rowstoadd = max - dfarr[i].shape[0]
    if(rowstoadd > 0):
        pad = np.zeros((rowstoadd,dfarr[i].shape[1]))
        dfarrc[i] = np.append(dfarrc[i], pad, axis = 0)
for i in range(len(dfarrc)):
    print(dfarrc[i].shape)

(20, 1296)
(287, 1296)
(200, 1296)
(237, 1296)
287
(287, 1296)
(287, 1296)
(287, 1296)
(287, 1296)


In [12]:
dfarrc
finalarray = dfarrc[0]

print(finalarray.shape)
for i in range(len(dfarrc)-1):
    i = i+1
    print(i)
    dfarrc[i] = dfarrc[i].reshape(1,287,1295)
    finalarray = np.stack((finalarray,dfarrc[i]))
finalarray.shape

(287, 1296)
1


ValueError: cannot reshape array of size 371952 into shape (1,287,1295)

In [13]:
finalarray = np.stack((dfarrc[0],dfarrc[1],dfarrc[2],dfarrc[3]))
finalarray.shape

(4, 287, 1296)

In [ ]:
!pip3 install torch

     |███████████████████▏            | 465.1 MB 56.9 MB/s eta 0:00:06    |█                               | 26.5 MB 21.9 MB/s eta 0:00:35     |█▌                              | 35.7 MB 21.9 MB/s eta 0:00:34     |██                              | 46.9 MB 49.1 MB/s eta 0:00:15     |██▊                             | 67.2 MB 49.1 MB/s eta 0:00:15     |███                             | 74.0 MB 49.1 MB/s eta 0:00:15     |███▍                            | 80.9 MB 49.1 MB/s eta 0:00:15     |████▌                           | 110.0 MB 15.6 MB/s eta 0:00:43     |████▊                           | 114.4 MB 15.6 MB/s eta 0:00:43     |█████▊                          | 138.0 MB 12.8 MB/s eta 0:00:50     |██████▍                         | 155.6 MB 12.8 MB/s eta 0:00:49     |██████▌                         | 157.8 MB 12.8 MB/s eta 0:00:49     |██████▉                         | 165.3 MB 12.8 MB/s eta 0:00:48     |███████▏                        | 174.0 MB 15.2 MB/s eta 0:00:40     |███████▌             

In [14]:
m.to_csv("microbio.csv")

In [17]:
k.to_csv("results.csv")

In [ ]:
import torch #pytorch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
import tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

model = Sequential()
model.add(LSTM(50, input_shape=(finalarray.shape[1], finaarray.shape[2])))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')
# fit network
history = model.fit(finalarray, m, epochs=50, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
# plot history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()